In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys



In [2]:
chromedriver = "/Users/snouto/Downloads/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
def get_movie_info(driver,url):
    driver.get(url)
    results = {}
    bs = BeautifulSoup(driver.page_source,"html5lib")
    genre_element = bs.select("div.facts > span.genres > a")
    user_rating_percentage = bs.find("div",class_="user_score_chart")
    section = bs.find("section",class_="facts left_column")
    if len(genre_element) > 0:
        genres = [x.text for x in genre_element]
        results['genre_count'] = str(len(genres))
        results['genre'] = ",".join(genres)
    if user_rating_percentage:
        results['rating'] = user_rating_percentage.get('data-percent')
    paragraphs = section.select("p")
    keywords_section = bs.find("section",class_="keywords right_column")
    keywords = keywords_section.select("li")
    results['keywords'] = str(len(keywords))
    for p in paragraphs:
        if "Revenue" in p.text:
            results['revenue'] = p.text.replace("Revenue","").replace("$","")
        if "Budget" in p.text:
            results['budget'] = p.text.replace("Budget","").replace("$","")

    return results

In [4]:
movies = defaultdict(dict)
url = "https://www.themoviedb.org/movie"
base_url = "https://www.themoviedb.org"
output_file = "/Users/snouto/Documents/movies/movies2.tsv"
num_scrolls = 100
driver = webdriver.Chrome(chromedriver)
driver.get(url)
for _ in range(num_scrolls):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    
        
        

In [5]:
bs = BeautifulSoup(driver.page_source, "html5lib")
elements = bs.find_all("div", class_="card style_1")
for element in elements:
    movie = {}
    movie_element = element.find("a", class_="image")
    title = movie_element.get('title')
    url = movie_element.get('href')
    if title in movies:
        # don't add duplicates
        continue
    movies[title]['title'] = title
    movies[title]['url'] = f"{base_url}{url}"
    # element.find("div",class_="content").find("p")
    release_date = element.select("div.content > p")
            # return list of one element (releasa_date)
    if len(release_date) > 0:
        movies[title]['release_date'] = release_date[0].text
print(f"Downloaded : {len(movies.keys())} Movies. Extending Them")

Downloaded : 99 Movies. Extending Them


In [6]:
# download more Details for movies
for title, movie in movies.items():
    print(f"Bringing information for movie: {title}")
    movie_details = get_movie_info(driver,movie['url'])
    movies[title].update(movie_details)

Bringing information for movie: Venom: Let There Be Carnage
Bringing information for movie: Free Guy
Bringing information for movie: Venom
Bringing information for movie: Snake Eyes: G.I. Joe Origins
Bringing information for movie: Deathstroke: Knights & Dragons - The Movie
Bringing information for movie: The Addams Family 2
Bringing information for movie: Cosmic Sin
Bringing information for movie: The Many Saints of Newark
Bringing information for movie: The Last Warrior: Root of Evil
Bringing information for movie: The Seven Deadly Sins: Cursed by Light
Bringing information for movie: The Stronghold
Bringing information for movie: Dragon Fury
Bringing information for movie: Solitary
Bringing information for movie: Shang-Chi and the Legend of the Ten Rings
Bringing information for movie: The Suicide Squad
Bringing information for movie: Crazy Fist
Bringing information for movie: F9
Bringing information for movie: Demon Slayer -Kimetsu no Yaiba- The Movie: Mugen Train
Bringing informat

In [7]:
# save data to Tab Separated Values
print(f"Saving Results to the output file {output_file}.")
with open(output_file,"w") as writer:
    keys = list(movies.keys())
    headers = list(movies[keys[0]].keys())
    headers = "\t".join(headers)
    writer.write(headers+"\n")
    for title , movie in movies.items():
        row = list(movie.values())
        writer.write("\t".join(row)+"\n")

print("results saved...")

Saving Results to the output file /Users/snouto/Documents/movies/movies2.tsv.
results saved...
